# Building Your First AI Agent 🤖

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/1337-Artificial-Intelligence/hackai-2025/blob/main/new_notebooks/agents_smolagents_toolcalling.ipynb)

In this notebook, you'll learn how to create an AI agent that can:
- Search the internet
- Do math calculations
- Check weather
- Get information about research papers

Time to complete: ~45 minutes

## What is an LLM Agent?

> ### **Common LLM Limitations:**
1. **Hallucination**
   - LLMs can generate incorrect information with high confidence
<img src="https://postimage.me/images/2025/05/21/image053efba357c12ca0.png" width=600/>

>2. **Knowledge Cutoff**
   - LLMs are limited to information from their training data
   - They cannot access or learn from information after their training period
<img src="https://postimage.me/images/2025/05/21/imageddfa1a7dc129c5f7.png" width=600/>

>3. **Data Privacy**
   - LLMs can only access public training data
   - They cannot access proprietary or private information
<img src="https://postimage.me/images/2025/05/21/imagee55206d53db62592.png" width=600/>

> ### **The Solution: LLM Agents**
An LLM Agent enhances a basic LLM by combining three key components:
- **LLM**: The core language model
- **Tools**: External capabilities and functions
- **Memory**: Ability to store and recall information

<img src="https://postimage.me/images/2025/05/21/image94e8ac07ffa04191.png" width=600/>

> This combination helps overcome the limitations by:
- Using tools to verify information and access external data
- Maintaining memory of past interactions and information
- Enabling access to current data through external tools

## How Does it Work?

> The LLM agent follows a structured decision-making process when responding to instructions. This process consists of three main steps that repeat until the task is complete:
>1. **Think/Planning:** The agent analyzes the user's request and formulates a step-by-step plan to accomplish the task.
>2. **Action:** The agent executes specific actions by calling appropriate tools with the necessary parameters.
>3. **Observation:** The agent evaluates the results of its actions and determines the next steps based on the outcomes.

<img src="https://postimage.me/images/2025/05/21/image2d6a0fed071fa281.png" width=600/>

>This cycle continues until the agent successfully completes the requested task or determines it cannot proceed further.

# Build Your First Agent Using Smolagent 🤗

Read more about smolagent:
* [**smolagents github**](https://github.com/huggingface/smolagents)
* [**smolagen course**](https://huggingface.co/learn/agents-course/en/unit2/smolagents/introduction)

> Let's build a sample agent that can:
* **Search the internet for information**
* **Do math calculations (+,-,*,/)**
* **Check the weather of a city**
* **arxiv paper info**

<img src="https://postimage.me/images/2025/05/21/imaged3e2447c5da0c4bb.png" width=600/>

#### 1. install smolagent library

In [ ]:
! uv pip install -U smolagents

#### 2. Select Your LLM Model

<img src="https://postimage.me/images/2025/05/21/imagecc657ebaccc20d06.png" width=600/>


>`smolagent` supports multiple LLM providers including OpenAI, Google, and HuggingFace. For this tutorial, we'll use Google's Gemini `gemini-2.0-flash` model through their free API.

>To get started:
1. Visit [Google AI Studio](https://aistudio.google.com/apikey)
2. Create an account if you don't have one
3. Generate an API key
4. Keep your API key secure - we'll use it in the next step.

>You can check **other providers** [here](https://github.com/huggingface/smolagents).

In [ ]:
from smolagents import OpenAIServerModel

# Load the model
model = OpenAIServerModel(
    model_id="gemini-2.0-flash",
    api_base="https://generativelanguage.googleapis.com/v1beta",
    api_key="",
)

In [ ]:
# test the model
response=model.client.chat.completions.create(
    model=model.model_id,
    messages=[
        {
            "role": "user",
            "content": "What is the capital of Morocco?",
        }
    ]
).choices[0].message.content
print(response)

The capital of Morocco is **Rabat**.



#### 3. Define Tools

>smolagents supports multiple tool types:
- LangChain tools
- MCP (Model Control Protocol) tools
- HuggingFace Space as tool

>In this example, we'll demonstrate how to:
1. Create custom tools (using `@tool` or inherit from `Tool`)
2. Use predefined tools from `smolagents` and `langchain`

>**Find out more langchain** [tools](https://python.langchain.com/docs/integrations/tools/).

<img src="https://postimage.me/images/2025/05/21/image367e87ae89c501d4.png" width=600/>


In [ ]:
from smolagents import Tool

class CalculatorTool(Tool):
    # the name of the tool should be clear.
    name="calculator"
    # the tool should have a description clearly stating what it does.
    description="A calculator that can perform basic arithmetic operations."
    # the tool should have a list of inputs and their types.
    inputs={
        "expression": {
            "type": "string",
            "description": "The arithmetic expression to calculate.",
        }
    }
    # the tool should have a of outputs and their types.
    output_type="string"
    # the tool should have forward method consisting of the logic of the tool.
    def forward(self, expression: str) -> str:
        try:
            result = eval(expression)
            return str(result)
        except Exception as e:
            return f"Error: {str(e)}"

# test CalculatorTool
calculator_tool = CalculatorTool()
response = calculator_tool("2 + 2 * 3")
print(response)

8


In [ ]:
from smolagents import tool

@tool
def city_weather(city: str) -> str:
    # the tool should have a clear name. and a string documnetation.
    # string documentation has description of the tool and its inputs.
    """
    Get the weather for a given city.
    Args:
        city: The name of the city.
    """
    # In a real-world scenario, this function would call a weather API.
    # For this example, we'll just return a dummy response.
    return f"The weather in {city} is sunny with a high of 25°C."

# test city_weather
response = city_weather("Benguerir")
print(response)

The weather in Benguerir is sunny with a high of 25°C.


In [ ]:
from smolagents import DuckDuckGoSearchTool

# Create a DuckDuckGo search tool
search_tool = DuckDuckGoSearchTool()
# Test Tool
response = search_tool("What is the capital of Morocco?")
print(response)

## Search Results

[Morocco | History, Map, Flag, Capital, People, & Facts | Britannica](https://www.britannica.com/place/Morocco)
Morocco is a mountainous country in western North Africa with a long history and diverse culture. Its capital is Rabat, located on the Atlantic coast, and its largest city is Casablanca.

[What Is The Capital Of Morocco? - WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-morocco.html)
Learn about the history, location, and role of Rabat as the political and cultural center of Morocco. Find out how Rabat became a capital city, what government it has, and what attractions it offers.

[Rabat - Wikipedia](https://en.wikipedia.org/wiki/Rabat)
Rabat is the capital city of Morocco and the country's seventh-largest city with an urban population of approximately 580,000. It was founded in the 12th century by the Almohads and has a rich history and cultural heritage.

[Morocco's Capital City, Map, & Historic Attractions - Britannica](https://www

In [ ]:
from langchain.agents import load_tools
from smolagents import Tool

arxiv_tool=Tool.from_langchain(load_tools(
    ["arxiv"]
)[0])

In [ ]:
result=arxiv_tool("1706.03762")
print(result)

Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 2 BLEU. On the WMT 2014 English-to-French
translation task, our model establishes a new sing

#### 4. Integrate Tools: Create Your Agent

In [ ]:
from smolagents import CodeAgent

tools=[city_weather,search_tool,calculator_tool,arxiv_tool]
agent=CodeAgent(
    model=model,
    tools=tools
)

#### 5. Run Your Agent

- If you want your agent to use memory you should to add `reset=False` when you run it.

<img src="https://postimage.me/images/2025/05/21/image97d3064f2d088e88.png" width=600/>

In [ ]:
query="""What's the paper 1706.03762 about,
and who is Noam Shazeer?
Also, what is the weather in Benguerir?
And what is 2 + 2 * 3?"""
response=agent.run(query)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the paper 1706.03762 about,                                                                              │
│ and who is Noam Shazeer?                                                                                        │
│ Also, what is the weather in Benguerir?                                                                         │
│ And what is 2 + 2 * 3?                                                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  paper_info = arxiv(query="1706.03762")                                                                           
  print(f"Paper information: {paper_info}")                                                                        
  noam_shazeer_info = web_search(query="Noam Shazeer")                                                             
  print(f"Noam Shazeer information: {noam_shazeer_info}")                                                          
  benguerir_weather = city_weather(city="Benguerir")                                                               
  print(f"Weather in Benguerir: {benguerir_weather}")                                                              
  calculation_result = calculator(expression="2 + 2 * 3")                                                          
  print(f"Calculation result: {calculation_result}")                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Paper information: Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, 
Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 2 BLEU. On the WMT 2014 English-to-French
translation task, our model establishes a new single-model state-of-the-art
BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction
of the training costs of the best models from the literature. We show that the
Transformer generalizes well to other tasks by applying it successfully to
English constituency parsing both with large and limited training data.
Noam Shazeer information: ## Search Results

[Noam Shazeer - Wikipedia](https://en.wikipedia.org/wiki/Noam_Shazeer)
Noam Shazeer is an American computer scientist and entrepreneur who developed transformer models and natural 
language processing. He co-founded Character.AI, which was acquired by Google in 2024, and co-leads the Gemini AI 
project.

[Noam Shazeer - Google | LinkedIn](https://www.linkedin.com/in/noam-shazeer-3b27288)
Noam Shazeer is a researcher and inventor of large language models at Google. He is also the co-founder of 
Character.ai, a company that builds personalized AI systems for human well-being.

[Noam Shazeer: The 100 Most Influential People in AI 2023 - 
TIME](https://time.com/collection/time100-ai/6310599/noam-shazeer/)
Before I interviewed Noam Shazeer, I interviewed his AI. Shazeer is the co-founder and CEO of Character.AI, a 
website that allows you to talk to AI versions of famous people, real and fictional ...

[‪Noam Shazeer‬ - ‪Google Scholar‬](https://scholar.google.com/citations?user=wsGvgA8AAAAJ&hl=en)
Noam Shazeer. Google. Verified email at google.com. Deep Learning. Articles Cited by Public access. Title. Sort. 
Sort by citations Sort by year Sort by title. Cited by. Cited by. ... N Shazeer, Y Cheng, N Parmar, D Tran, A 
Vaswani, P Koanantakool, ... Advances in neural information processing systems 31, 2018. 458:

[Noam Shazeer: After 20 years at Google, he walked away, then came back 
...](https://gulfnews.com/special-reports/noam-shazeer-after-20-years-at-google-he-walked-away-then-came-back-for-2
7-billion-1.1734670875849)
Learn about Noam Shazeer's journey from Google to Character.ai, a platform for conversational AI with customised 
personas. Discover his inventions, achievements, and challenges in the field of artificial intelligence.

[Google Rehires AI Pioneer Noam Shazeer in $2.7 Billion Deal - 
Entrepreneur](https://www.entrepreneur.com/business-news/google-rehires-ai-pioneer-noam-shazeer-in-27-billion-deal/
480378)
Noam Shazeer, who invented much of the current revolution in large language models, quit Google in 2021 after 
creating an AI chatbot the company refused to take public. He returned to Google as part of the DeepMind research 
team through a deal with Character.AI, a startup he founded.

[Noam Noam Shazeer Explains How He Co-Created the AI 
Transformer](https://www.deeplearning.ai/the-batch/ai-transformed/)
Noam Shazeer is a co-inventor of the transformer network, a breakthrough architecture for natural language 
processing. In this interview, he discusses his journey, his insights and his vision for the future of AI.

[Google h

[Step 1: Duration 6.15 seconds| Input tokens: 2,417 | Output tokens: 228]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Paper 1706.03762 is titled 'Attention Is All You Need' and is about the Transformer architecture   
  for sequence transduction. Noam Shazeer is an American computer scientist and entrepreneur who co-developed      
  transformer models and natural language processing, co-founded Character.AI, and co-leads the Gemini AI          
  project. The weather in Benguerir is sunny with a high of 25°C. 2 + 2 * 3 = 8.")                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Paper 1706.03762 is titled 'Attention Is All You Need' and is about the Transformer 
architecture for sequence transduction. Noam Shazeer is an American computer scientist and entrepreneur who 
co-developed transformer models and natural language processing, co-founded Character.AI, and co-leads the Gemini 
AI project. The weather in Benguerir is sunny with a high of 25°C. 2 + 2 * 3 = 8.

[Step 2: Duration 1.53 seconds| Input tokens: 6,721 | Output tokens: 358]

In [ ]:
from pprint import pprint
pprint(response)

("Paper 1706.03762 is titled 'Attention Is All You Need' and is about the "
 'Transformer architecture for sequence transduction. Noam Shazeer is an '
 'American computer scientist and entrepreneur who co-developed transformer '
 'models and natural language processing, co-founded Character.AI, and '
 'co-leads the Gemini AI project. The weather in Benguerir is sunny with a '
 'high of 25°C. 2 + 2 * 3 = 8.')


# Exercice

## Your Turn! 🎯

Try these challenges:
1. Ask the agent about a different research paper
2. Get the weather for another city
3. Try a more complex math problem
4. Ask it to search for something interesting

Remember: The more specific your questions, the better the answers!

## What's Next? 🚀

You've built your first AI agent! Here's what you can explore next:
- Add more tools to your agent
- Try different AI models
- Build agents for specific tasks
- Learn about prompt engineering

Keep experimenting and have fun! 🎉